**Task 08: Completing missing data**

In [2]:
!pip install rdflib
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4/"

     |████████████████████████████████| 235kB 2.4MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 


In [3]:
from rdflib import Graph, Namespace, Literal, URIRef
g1 = Graph()
g2 = Graph()
g1.parse(github_storage+"resources/data01.rdf", format="xml")
g2.parse(github_storage+"resources/data02.rdf", format="xml")

<Graph identifier=Nf1eb178ca4794534b3837f7f979a7d06 (<class 'rdflib.graph.Graph'>)>

Task: list all the elements of the class Person in the first graph (data01.rdf) and complete the fields (given name, family name and email) that may be missing with the data from the second graph (data02.rdf). You can use SPARQL queries or iterate the graph, or both.

First, we do a query to see a list with all the elements of the class Person in the first graph g1

In [4]:
from rdflib.plugins.sparql import prepareQuery
from rdflib.namespace import RDF, RDFS

NS = Namespace("http://data.org#")

q1 = prepareQuery('''
  SELECT DISTINCT ?elem
  WHERE
  {
    ?elem a/rdfs:subClassOf* ns:Person.
  }
  ''',
  initNs = {"ns": NS}
)

for s in g1.query(q1):
  print(s)

(rdflib.term.URIRef('http://data.org#SaraJones'),)
(rdflib.term.URIRef('http://data.org#JohnDoe'),)
(rdflib.term.URIRef('http://data.org#HarryPotter'),)
(rdflib.term.URIRef('http://data.org#JohnSmith'),)


Now, lets see the fields of our elements from g1 and save them on a list

In [5]:
list1 = []
for s, p, o in g1.triples((None,None,None)):
  list1.append(s + "*"+ p + "*" + o)
print(list1)

[rdflib.term.URIRef('http://data.org#JohnSmith*http://www.w3.org/1999/02/22-rdf-syntax-ns#type*http://data.org#Person'), rdflib.term.URIRef('http://data.org#JohnDoe*http://www.w3.org/2001/vcard-rdf/3.0#EMAIL*doe@data.org'), rdflib.term.URIRef('http://www.w3.org/2001/vcard-rdf/3.0#Given*http://www.w3.org/1999/02/22-rdf-syntax-ns#type*http://www.w3.org/2000/01/rdf-schema#Datatype'), rdflib.term.URIRef('http://data.org#SaraJones*http://www.w3.org/1999/02/22-rdf-syntax-ns#type*http://data.org#Person'), rdflib.term.URIRef('http://www.w3.org/2001/vcard-rdf/3.0#Family*http://www.w3.org/2000/01/rdf-schema#range*http://www.w3.org/2001/XMLSchema#string'), rdflib.term.URIRef('http://data.org#JohnSmith*http://www.w3.org/2001/vcard-rdf/3.0#EMAIL*j.smith@data.org'), rdflib.term.URIRef('http://data.org#JohnDoe*http://www.w3.org/2001/vcard-rdf/3.0#Family*Doe'), rdflib.term.URIRef('http://data.org#Person*http://www.w3.org/1999/02/22-rdf-syntax-ns#type*http://www.w3.org/2000/01/rdf-schema#Class'), rdfli

Lets do the same with de second graph g2

In [6]:
list2 = []
for s, p, o in g2.triples((None,None,None)):
  list2.append(s + "*" + p + "*" + o)
print(list2)

http://data.org#HarryPotter*http://www.w3.org/2001/vcard-rdf/3.0#FN*Harry Potter does not look like a valid URI, trying to serialize this will break.


[rdflib.term.URIRef('http://data.org#JohnSmith*http://www.w3.org/1999/02/22-rdf-syntax-ns#type*http://data.org#Person'), rdflib.term.URIRef('http://data.org#SaraJones*http://www.w3.org/2001/vcard-rdf/3.0#Family*Jones'), rdflib.term.URIRef('http://data.org#JohnDoe*http://www.w3.org/2001/vcard-rdf/3.0#Family*Doe'), rdflib.term.URIRef('http://www.w3.org/2001/vcard-rdf/3.0/Given*http://www.w3.org/1999/02/22-rdf-syntax-ns#type*http://www.w3.org/2000/01/rdf-schema#Datatype'), rdflib.term.URIRef('http://www.w3.org/2001/vcard-rdf/3.0/EMAIL*http://www.w3.org/2000/01/rdf-schema#range*http://www.w3.org/2001/XMLSchema#string'), rdflib.term.URIRef('http://www.w3.org/2001/vcard-rdf/3.0/Family*http://www.w3.org/2000/01/rdf-schema#range*http://www.w3.org/2001/XMLSchema#string'), rdflib.term.URIRef('http://www.w3.org/2001/vcard-rdf/3.0/FN*http://www.w3.org/2000/01/rdf-schema#range*http://www.w3.org/2001/XMLSchema#string'), rdflib.term.URIRef('http://data.org#JohnDoe*http://www.w3.org/1999/02/22-rdf-syn

Now, we are going to compare both lists and save the files wich are in g2 but
not in g1 in another list

In [7]:
list3 = []
for i in range(len(list2)):
  if(list2[i] not in list1):
    list3.append(list2[i].replace('*',' '))

We have discovered the missing data values, lets add them to g1

In [8]:
for i in range(len(list3)):
  tmp = list3[i].split()
  s = URIRef(tmp[0])
  p = URIRef(tmp[1])
  o = tmp[2] 
  # Be careful, some objects are rdflib.term.URIRef but others could be 
  # rdflib.term.Literal. We should distinguish between both cases
  if 'http' in o:
    o = URIRef(tmp[2])
  else:
    o = Literal(tmp[2]) 
  g1.add((s, p, o))


Finally, we can see the full graph serialized with the next command

In [9]:
print(g1.serialize(format="ttl").decode("UTF-8"))

@prefix : <http://data.org#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix vcard-rdf: <http://www.w3.org/2001/vcard-rdf/3.0#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:Person a rdfs:Class .

:HarryPotter a :Person ;
    vcard-rdf:EMAIL "hpotter@hogwarts.org" ;
    vcard-rdf:FN "Harry" ;
    vcard-rdf:Family "Potter"^^xsd:string ;
    vcard-rdf:Given "Harry"^^xsd:string .

:JohnDoe a :Person ;
    vcard-rdf:EMAIL "doe@data.org"^^xsd:string ;
    vcard-rdf:Family "Doe"^^xsd:string ;
    vcard-rdf:Given "John" .

:JohnSmith a :Person ;
    vcard-rdf:EMAIL "j.smith@data.org"^^xsd:string ;
    vcard-rdf:Family "Smith" ;
    vcard-rdf:Given "John"^^xsd:string .

:SaraJones a :Person ;
    vcard-rdf:EMAIL "sara.jones@data.org" ;
    vcard-rdf:Family "Jones" ;
    vcard-rdf:Given "Sara" .

vcard-rdf:EMAIL a rdfs:Datatype ;
    rdfs:range xsd:string .

vcard-rdf:Family a rdfs:Property ;
    rdfs:range xsd:string .

vcard-rdf:Given a rdfs:Datatype ;
    rdfs:rang